In [ ]:
import robotic as ry
import numpy as np

from time import sleep





C = ry.Config()

maps = ["map1_simple_cross.g", "map2_tunnel.g", "map3_two_cave.g"]


C.addFile(f"maps/{maps[2]}")

C.view()




0

NO SELECTION: SELECTION DEPTH = 1 200
